# Extracting Data

Extracting data from source database using API from kaggle

In [ ]:
stop here execute from 2nd line

In [6]:
import opendatasets as od

dataset='https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce'
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ankit117kr
Your Kaggle Key: ········


100%|██████████| 42.6M/42.6M [00:15<00:00, 2.96MB/s]


List of all file downloaded from source dataset

In [15]:
import os

data_dir='./brazilian-ecommerce'

print('total',len(os.listdir(data_dir)),'file downloaded from the source database as listed below')
os.listdir(data_dir)

total 9 file downloaded from the source database as listed below


['.ipynb_checkpoints',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

Selecting one table at one time and performing EDA

In [22]:
import pandas as pd

df= pd.read_csv('./brazilian-ecommerce/product_category_name_translation.csv')
df.sample(5)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


Exploratory Data Analysis of tables 

a) Shape and size of DataFrame

b) Finding Dublicate rows, Missing Values, Outlier value

c) Measure of central tendency (mean, median, mode)

d) Kernel density function (skewness, kurtosis)

e) Measure of dispersion (standard deviation, Range, min, max, percentile)

In [31]:
df.shape

(71, 2)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [25]:
df.describe()

,product_category_name,product_category_name_english
count,71,71
unique,71,71
top,beleza_saude,health_beauty
freq,1,1


In [27]:
df.isnull().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

uploading the dataset into SQL server or local server to built relationship among table and organising dataset

# Transforming Data

Loading data from local server and performing Feature Engineering as per requirement of matrix

In [51]:
import pyodbc

conn = pyodbc.connect("Driver={SQL Server};"
                      "Server=LAPTOP-BTANJQA0\\ankro,1433;" 
                      "database=E_COMMERCE;" 
                      "Trusted_connection=yes;")

cursr=conn.cursor()

Writing query as per KPI metric

In [129]:
cursr.execute('WITH CTE_Table AS('
        'SELECT  P.Category_ID,P.Product,SUM(OD.Quantity)AS Total_sold,DENSE_RANK()OVER(PARTITION BY Category_ID ORDER BY SUM(OD.Quantity) DESC) AS Rnk '
        'FROM Category AS C JOIN Products AS P ON C.CategoryID=P.Category_ID JOIN OrderDetails AS OD ON P.ProductID=OD.ProductID JOIN Orders AS O ON OD.OrderID=O.OrderID ' 
        'WHERE C.Active = \'Yes\' '
        'GROUP BY P.Category_ID,P.Product)'
        'SELECT Category_ID,Product,Total_sold, Rnk FROM CTE_Table '
        'WHERE Rnk<=3 GROUP BY Category_ID,Product,Total_sold,Rnk ORDER BY Category_ID,Rnk;')

data = cursr.fetchall()

In [130]:
col_names = [desc[0] for desc in cursr.description]
KPI1 = pd.DataFrame.from_records(data, columns=col_names)
KPI1

,Category_ID,Product,Total_sold,Rnk
0,5001,Della Storage Container With Lid - Assorted Co...,113,1
1,5001,"Dyna Plastic Bathroom Stool/Patla - Printed, A...",95,2
2,5001,Toilet Cleaner,93,3
3,5002,Pantry/Cookie/Snacks Glass Jar With Blue Lid,103,1
4,5002,Storage Steel Airtight Round Container - Klip ...,95,2
5,5002,"Adult Dry Cat Food - +1 Year, Ocean Fish",78,3
6,5003,Turmeric Powder/Arisina Pudi,269,1
7,5003,Ghee/Tuppa,188,2
8,5003,Coriander Powder,186,3
9,5004,Extra Virgin Olive Oil,194,1


1)  KPI for Average delivery time for each Vendor company

In [131]:
cursr.execute('SELECT S.CompanyName, AVG(DATEDIFF(DD, O.ShipDate, O.DeliveryDate)) AS Delivery_Time '
              'FROM Orders AS O JOIN Shippers AS S ON O.ShipperID = S.ShipperID ' 
              'GROUP BY S.CompanyName ' )

data = cursr.fetchall()

In [132]:
col_names = [desc[0] for desc in cursr.description]
KPI2 = pd.DataFrame.from_records(data, columns=col_names)
KPI2

,CompanyName,Delivery_Time
0,Blue Dart,10
1,COSCO China Ocean Shipping Company,10
2,Delhivery,10
3,DTDC,10
4,Fed Ex,10
5,Hapag Lloyd,10
6,Lufthansa Cargo,10
7,ONE Ocean Network Express,10


Feature Engineering

a) Handeling Missing Value, Dublicate row

b) Handeling Outlier Data

c) Handeling Category&Numerical Data

d) Feature extraction

e) Feature Scaling